In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


text = """It was the best of times, it was the worst of times, it was the age of
wisdom, it was the age of foolishness, it was the epoch of belief, it was
the epoch of incredulity, it was the season of Light, it was the season of
Darkness, it was the spring of hope, it was the winter of despair, we had
everything before us, we had nothing before us, we were all going direct
to Heaven, we were all going direct the other way—in short, the period
was so far like the present period, that some of its noisiest authorities
insisted on its being received, for good or for evil, in the superlative
degree of comparison only."""
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

words = word_tokenize(text)
words = [w for w in words if not w.lower() in stop_words]
words = [lemmatizer.lemmatize(w) for w in words]

words

['best',
 'time',
 ',',
 'worst',
 'time',
 ',',
 'age',
 'wisdom',
 ',',
 'age',
 'foolishness',
 ',',
 'epoch',
 'belief',
 ',',
 'epoch',
 'incredulity',
 ',',
 'season',
 'Light',
 ',',
 'season',
 'Darkness',
 ',',
 'spring',
 'hope',
 ',',
 'winter',
 'despair',
 ',',
 'everything',
 'u',
 ',',
 'nothing',
 'u',
 ',',
 'going',
 'direct',
 'Heaven',
 ',',
 'going',
 'direct',
 'way—in',
 'short',
 ',',
 'period',
 'far',
 'like',
 'present',
 'period',
 ',',
 'noisiest',
 'authority',
 'insisted',
 'received',
 ',',
 'good',
 'evil',
 ',',
 'superlative',
 'degree',
 'comparison',
 '.']